In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate
# Get data from wikipedia
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'lxml')

# Look for the table in the source html with class wikitable sortable and read all the rows in a list 
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])
    

### create a data frame with columns 'PostalCode', 'Borough', 'Neighbourhood' fill it with the data as retrieved 

In [2]:
# create a data frame with columns 'PostalCode', 'Borough', 'Neighbourhood' fill it with the data as retrieved 
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows
df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Remove all the rows where Borough is Not Assign

In [3]:
# Remove all the rows where Borough is Not Assign
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

df1=df.reset_index()
df1.head()

,index,PostalCode,Borough,Neighbourhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,6,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Joining Neighbourhoods having same postal codes and Assigning Borough to the "Not assigned" Neighbourhoods

In [4]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
df3=df2.reset_index()
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
df3.shape

(103, 3)